In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Put your statements here
import pandas as pd
import numpy as np

In [4]:
#import the input data
csv_path = '/content/drive/MyDrive/Colab Notebooks/Flink/nation.csv'
nation = pd.read_csv(csv_path, sep='|',header=None)
nation.head()
nation.columns = ['NATIONKEY','n_name','n','n','n']
nation = nation[['NATIONKEY','n_name']]
nation.head()
csv_path1 = '/content/drive/MyDrive/Colab Notebooks/Flink/customer.csv'
customer = pd.read_csv(csv_path1, sep='|',header=None)
customer.head()
customer.columns = ['CUSTKEY','c_name','c_address','NATIONKEY','c_phone','c_acctbal','n','c_comment','n1']
customer = customer.drop(['n','n1'],axis = 'columns')
customer.head()

,CUSTKEY,c_name,c_address,NATIONKEY,c_phone,c_acctbal,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2988,711.56,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3665,121.65,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7498.12,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2866.83,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,794.47,n accounts will have to unwind. foxes cajole a...


In [5]:
csv_path2 = '/content/drive/MyDrive/Colab Notebooks/Flink/orders.csv'
orders = pd.read_csv(csv_path2, sep='|',header=None)
orders.columns = ['ORDERKEY','CUSTKEY','n','n','o_orderdate','n','n','n','n','n']
orders = orders[['ORDERKEY','CUSTKEY','o_orderdate']]
orders

,ORDERKEY,CUSTKEY,o_orderdate
0,1,36901,1996-01-02
1,2,78002,1996-12-01
2,3,123314,1993-10-14
3,4,136777,1995-10-11
4,5,44485,1994-07-30
...,...,...,...
1499995,5999972,143594,1996-05-02
1499996,5999973,32071,1997-07-13
1499997,5999974,55448,1993-07-28
1499998,5999975,113398,1993-07-25


In [6]:
csv_path3 = '/content/drive/MyDrive/Colab Notebooks/Flink/lineitem.csv'
lineitem = pd.read_csv(csv_path3, sep='|',header=None)
lineitem.head()
lineitem.columns = ['ORDERKEY','n','n','n','n','l_extendedprice','l_discount','n','l_returnflag','n','n','n','n','n','n','n','n']
lineitem = lineitem[['ORDERKEY','l_extendedprice','l_discount','l_returnflag']]
lineitem

,ORDERKEY,l_extendedprice,l_discount,l_returnflag
0,1,21168.23,0.04,N
1,1,45983.16,0.09,N
2,1,13309.60,0.10,N
3,1,28955.64,0.09,N
4,1,22824.48,0.10,N
...,...,...,...,...
6001210,5999975,37736.64,0.07,R
6001211,5999975,9509.15,0.04,A
6001212,5999975,19226.34,0.04,A
6001213,6000000,5936.25,0.04,N


In [7]:
def split(input_data):
  "Returns the 25%,50%,75%,100% of the input_data"
  data_split = np.array_split(input_data, 4)
  p1, p2, p3, p4 = pd.DataFrame(data_split[0]),pd.DataFrame(data_split[1]),pd.DataFrame(data_split[2]),pd.DataFrame(data_split[3])
  p25 = p1
  p50 = pd.concat([p1,p2])
  p75 = pd.concat([p1,p2,p3])
  p100 = pd.concat([p1,p2,p3,p4])
  return p25, p50, p75, p100

In [8]:
#split the data into 4 parts
nation25,nation50,nation75,nation100 = split(nation)
customer25,customer50,customer75,customer100 = split(customer)
orders25,orders50,orders75,orders100 = split(orders)
lineitem25,lineitem50,lineitem75,lineitem100 = split(lineitem)

In [9]:
def SQLJOIN(nation,customer,orders,lineitem):
  "Returns the join result"
  #clean nation based on condition
  nation_cleaned = nation.loc[nation['n_name']=='CANADA']
  #join nation and customer
  nation_customer = pd.merge(nation_cleaned, customer, on='NATIONKEY', how='inner')
  #conditions on orders' date
  date = "1995-02-06"
  orders_date = orders.loc[orders['o_orderdate']==date]
  #joining nation and customer and orders
  nation_cust_order = pd.merge(nation_customer, orders_date, on='CUSTKEY', how='inner')
  #conditions on orders' date
  lineitem_new = lineitem.loc[lineitem['l_returnflag']=='R']
  result = pd.merge(nation_cust_order, lineitem_new, on='ORDERKEY', how='inner')
  return result

In [10]:
res25 = SQLJOIN(nation25,customer25,orders25,lineitem25)
res50 = SQLJOIN(nation50,customer50,orders50,lineitem50)
res75 = SQLJOIN(nation75,customer75,orders75,lineitem75)
res100 = SQLJOIN(nation100,customer100,orders100,lineitem100)
res25 = res25.sample(frac = 1)
res25

,NATIONKEY,n_name,CUSTKEY,c_name,c_address,c_phone,c_acctbal,c_comment,ORDERKEY,o_orderdate,l_extendedprice,l_discount,l_returnflag
2,3,CANADA,233,Customer#000000233,mFm45wZ7rV4VIbEE1F4,13-574-104-3221,3998.24,st the special instructions. theodolites detec...,428421,1995-02-06,23249.10,0.01,R
6,3,CANADA,26603,Customer#000026603,"g0YS23E8QO RgL6wNBpH,5lpTOjmN3zVB0r",13-355-953-1372,-219.99,"e blithely stealthily final packages? even, ir...",566560,1995-02-06,32996.49,0.06,R
5,3,CANADA,8305,Customer#000008305,2CETp2L7SGCLjko3YZgYeph4Se,13-683-821-6803,4350.44,nag slyly among the unusual accounts. slyly regu,838533,1995-02-06,12388.11,0.08,R
1,3,CANADA,233,Customer#000000233,mFm45wZ7rV4VIbEE1F4,13-574-104-3221,3998.24,st the special instructions. theodolites detec...,428421,1995-02-06,35860.23,0.02,R
0,3,CANADA,233,Customer#000000233,mFm45wZ7rV4VIbEE1F4,13-574-104-3221,3998.24,st the special instructions. theodolites detec...,428421,1995-02-06,73212.88,0.02,R
3,3,CANADA,8305,Customer#000008305,2CETp2L7SGCLjko3YZgYeph4Se,13-683-821-6803,4350.44,nag slyly among the unusual accounts. slyly regu,838533,1995-02-06,41744.63,0.05,R
7,3,CANADA,26603,Customer#000026603,"g0YS23E8QO RgL6wNBpH,5lpTOjmN3zVB0r",13-355-953-1372,-219.99,"e blithely stealthily final packages? even, ir...",566560,1995-02-06,6374.56,0.05,R
4,3,CANADA,8305,Customer#000008305,2CETp2L7SGCLjko3YZgYeph4Se,13-683-821-6803,4350.44,nag slyly among the unusual accounts. slyly regu,838533,1995-02-06,22773.72,0.10,R


**Evaluation**   
The following code cell compares the result generated by Flink with the ground truth generated by pandas.

In [13]:
#group by and aggregation 25% of data
result = res25
final = (result
     .assign(temp=result.eval('l_extendedprice*(1-l_discount)'))
     .groupby(["CUSTKEY", "c_name",'c_acctbal','c_phone','n_name'])
     .sum()
     .eval('temp')
     .to_frame('revenue')
)
#the output performed using original datafiles
final

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326


In [14]:
#the output generated by Flink
csv_path = '/content/drive/MyDrive/Colab Notebooks/Flink/data25_res.csv'
soln = pd.read_csv(csv_path,header=None)
soln.columns = ['c_custkey','c_name','revenue','c_acctbal','n_name']
#group by
final_result = soln.groupby(['c_custkey','c_name','c_acctbal',
                             'n_name'])['revenue'].max()
final_result = pd.DataFrame(final_result)
final_result

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568


In [15]:
#group by and aggregation 25% of data
result = res50
final = (result
     .assign(temp=result.eval('l_extendedprice*(1-l_discount)'))
     .groupby(["CUSTKEY", "c_name",'c_acctbal','c_phone','n_name'])
     .sum()
     .eval('temp')
     .to_frame('revenue')
)
#the output performed using original datafiles
final

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
15139,Customer#000015139,5601.01,13-900-889-3875,CANADA,203727.1886
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326
33028,Customer#000033028,6030.33,13-694-879-1038,CANADA,73122.3180


In [17]:
#the output generated by Flink
csv_path = '/content/drive/MyDrive/Colab Notebooks/Flink/data50_res.csv'
soln = pd.read_csv(csv_path,header=None)
soln.columns = ['c_custkey','c_name','revenue','c_acctbal','n_name']
#group by
final_result = soln.groupby(['c_custkey','c_name','c_acctbal',
                             'n_name'])['revenue'].max()
final_result = pd.DataFrame(final_result)
final_result

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326
15139,Customer#000015139,5601.01,13-900-889-3875,CANADA,203727.1886
33028,Customer#000033028,6030.33,13-694-879-1038,CANADA,73122.3180
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568


In [12]:
#group by and aggregation
result = res75
final = (result
     .assign(temp=result.eval('l_extendedprice*(1-l_discount)'))
     .groupby(["CUSTKEY", "c_name",'c_acctbal','c_phone','n_name'])
     .sum()
     .eval('temp')
     .to_frame('revenue')
)
#the output performed using original datafiles
final

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
15139,Customer#000015139,5601.01,13-900-889-3875,CANADA,203727.1886
20653,Customer#000020653,549.69,13-429-873-7900,CANADA,103444.2572
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326
33028,Customer#000033028,6030.33,13-694-879-1038,CANADA,73122.3180
97922,Customer#000097922,760.16,13-449-575-8264,CANADA,3007.9104
102763,Customer#000102763,2232.37,13-137-890-3251,CANADA,53280.6504


In [67]:
#the output generated by Flink
csv_path = '/content/drive/MyDrive/Colab Notebooks/Flink/data75_res.csv'
soln = pd.read_csv(csv_path,header=None)
soln.columns = ['c_custkey','c_name','revenue','c_acctbal','n_name']
#group by
final_result = soln.groupby(['c_custkey','c_name','c_acctbal',
                             'n_name'])['revenue'].max()
final_result = pd.DataFrame(final_result)
final_result

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
97922,Customer#000097922,760.16,13-449-575-8264,CANADA,3007.9104
33028,Customer#000033028,6030.33,13-694-879-1038,CANADA,73122.3180
20653,Customer#000020653,549.69,13-429-873-7900,CANADA,103444.2572
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
15139,Customer#000015139,5601.01,13-900-889-3875,CANADA,203727.1886
102763,Customer#000102763,2232.37,13-137-890-3251,CANADA,53280.6504
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568


In [18]:
#group by and aggregation
result = res100
final = (result
     .assign(temp=result.eval('l_extendedprice*(1-l_discount)'))
     .groupby(["CUSTKEY", "c_name",'c_acctbal','c_phone','n_name'])
     .sum()
     .eval('temp')
     .to_frame('revenue')
)
#the output performed using original datafiles
final

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568
1717,Customer#000001717,1715.40,13-450-115-4347,CANADA,91060.0880
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
15139,Customer#000015139,5601.01,13-900-889-3875,CANADA,203727.1886
20653,Customer#000020653,549.69,13-429-873-7900,CANADA,103444.2572
26603,Customer#000026603,-219.99,13-355-953-1372,CANADA,37072.5326
33028,Customer#000033028,6030.33,13-694-879-1038,CANADA,73122.3180
53720,Customer#000053720,4988.41,13-877-286-4642,CANADA,189109.0720
65647,Customer#000065647,524.48,13-743-299-8123,CANADA,55312.2080


In [19]:
#the output generated by Flink
csv_path = '/content/drive/MyDrive/Colab Notebooks/Flink/data100_res.csv'
soln = pd.read_csv(csv_path,header=None)
soln.columns = ['c_custkey','c_name','revenue','c_acctbal','n_name']
#group by
final_result = soln.groupby(['c_custkey','c_name','c_acctbal',
                             'n_name'])['revenue'].max()
final_result = pd.DataFrame(final_result)
final_result

,,,,,revenue
CUSTKEY,c_name,c_acctbal,c_phone,n_name,
132082,Customer#000132082,1794.84,13-155-693-5984,CANADA,53303.9052
141118,Customer#000141118,4469.48,13-487-761-6272,CANADA,143555.1963
8305,Customer#000008305,4350.44,13-683-821-6803,CANADA,71550.8077
233,Customer#000000233,3998.24,13-574-104-3221,CANADA,129908.2568
102763,Customer#000102763,2232.37,13-137-890-3251,CANADA,53280.6504
33028,Customer#000033028,6030.33,13-694-879-1038,CANADA,73122.3180
53720,Customer#000053720,4988.41,13-877-286-4642,CANADA,189109.0720
140752,Customer#000140752,238.27,13-236-186-5586,CANADA,93791.2192
132163,Customer#000132163,4522.34,13-502-886-4597,CANADA,40355.5680
